In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
param_grid={'weights':["uniform", "distance", None], 'algorithm': ["auto", "ball_tree","kd_tree","brute"], 'leaf_size': [10,20,30], 'p': [1,2,3,4,5]}
grid= GridSearchCV(KNeighborsClassifier(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


C:\Anaconda3run\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [10, 20, 30], 'p': [1, 2, 3, 4, 5],
                         'weights': ['uniform', 'distance', None]},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred=grid.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[78,  7],
       [ 4, 45]], dtype=int64)

In [14]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [15]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [16]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.9406962785114046

In [17]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'algorithm': 'auto', 'leaf_size': 10, 'p': 2, 'weights': 'uniform'}


In [18]:
grid.cv_results_

{'mean_fit_time': array([0.01216793, 0.00578537, 0.00478129, 0.00637722, 0.00538507,
        0.00638547, 0.0069818 , 0.00697002, 0.00698943, 0.00718279,
        0.00777965, 0.00637774, 0.00418015, 0.00619068, 0.00498614,
        0.00498199, 0.00698113, 0.00738106, 0.00478706, 0.00598435,
        0.00598526, 0.00498643, 0.00618601, 0.00678205, 0.0045897 ,
        0.00677776, 0.00658255, 0.00558372, 0.00737786, 0.00778356,
        0.00478673, 0.00558815, 0.00558529, 0.00458789, 0.00718188,
        0.00558367, 0.00578508, 0.00538936, 0.00717869, 0.00858021,
        0.00658288, 0.00518703, 0.00438061, 0.003791  , 0.00418811,
        0.00498748, 0.00718355, 0.00698214, 0.01036882, 0.00478635,
        0.01017179, 0.00897546, 0.0065845 , 0.00718007, 0.0079752 ,
        0.00698099, 0.0045855 , 0.00598421, 0.00538554, 0.00837946,
        0.0051888 , 0.004986  , 0.00598445, 0.00877619, 0.00538521,
        0.00558429, 0.0057847 , 0.00438747, 0.00478454, 0.00637994,
        0.00598578, 0.00618386,

In [19]:
re=grid.cv_results_

In [20]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012168,0.007176,0.023337,0.001954,auto,10,1,uniform,"{'algorithm': 'auto', 'leaf_size': 10, 'p': 1,...",0.847141,0.906252,0.850543,0.944161,0.981233,0.905866,0.052261,97
1,0.005785,0.000978,0.014563,0.001014,auto,10,1,distance,"{'algorithm': 'auto', 'leaf_size': 10, 'p': 1,...",0.867478,0.906252,0.833323,0.925272,0.962264,0.898918,0.044838,121
2,0.004781,0.001331,0.022339,0.002326,auto,10,1,None,"{'algorithm': 'auto', 'leaf_size': 10, 'p': 1,...",0.847141,0.906252,0.850543,0.944161,0.981233,0.905866,0.052261,97
3,0.006377,0.002053,0.020544,0.002570,auto,10,2,uniform,"{'algorithm': 'auto', 'leaf_size': 10, 'p': 2,...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
4,0.005385,0.001017,0.015160,0.002310,auto,10,2,distance,"{'algorithm': 'auto', 'leaf_size': 10, 'p': 2,...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.005587,0.001740,0.096740,0.005353,brute,30,4,distance,"{'algorithm': 'brute', 'leaf_size': 30, 'p': 4...",0.847141,0.906252,0.851527,0.944161,0.924528,0.894722,0.038975,145
176,0.004189,0.001163,0.115889,0.011378,brute,30,4,None,"{'algorithm': 'brute', 'leaf_size': 30, 'p': 4...",0.867478,0.906252,0.869709,0.944161,0.943699,0.906260,0.033699,25
177,0.005186,0.001163,0.100934,0.003590,brute,30,5,uniform,"{'algorithm': 'brute', 'leaf_size': 30, 'p': 5...",0.867478,0.906252,0.869709,0.944161,0.943699,0.906260,0.033699,25
178,0.004187,0.000746,0.098138,0.007740,brute,30,5,distance,"{'algorithm': 'brute', 'leaf_size': 30, 'p': 5...",0.847141,0.906252,0.851527,0.944161,0.924528,0.894722,0.038975,145


In [21]:
grid.best_score_

0.9137665476099139

In [22]:
pre_input= sc.transform([[35,60000,0]])
pre_input

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 6.90094695e+01,  5.91413422e+03, -2.05850391e+00]])

In [23]:
result= grid.predict(pre_input)
result

array([1], dtype=int64)